In [1]:
!pip install --no-deps PyPDF2 python-docx langchain langchain-community langchain-huggingface groq sentence-transformers pymupdf


In [9]:
!pip install langchain-groq
!pip install docx2txt
!pip install faiss-gpu
!pip install faiss-cpu


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


ERROR: Could not find a version that satisfies the requirement faiss-gpu (from versions: none)
ERROR: No matching distribution found for faiss-gpu


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 53.1 MB/s eta 0:00:00:00:0100:01


In [3]:
import os
import PyPDF2
import docx
import pandas as pd
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import SupabaseVectorStore
from langchain.schema import Document
from groq import Groq
from supabase import create_client, Client
import ipywidgets as widgets
from IPython.display import display, clear_output

In [4]:
import os
import traceback
import fitz
import pandas as pd
from pathlib import Path


In [ ]:
# Initialize Supabase client
SUPABASE_URL = ""
SUPABASE_KEY = ""
supabase: Client = create_client(SUPABASE_URL, SUPABASE_KEY)



In [ ]:
# Initialize Groq client
os.environ["GROQ_API_KEY"] = ""
client = Groq()


In [7]:
from langchain_community.document_loaders import (
    PyPDFLoader, TextLoader, CSVLoader, Docx2txtLoader
)
from langchain_core.documents import Document

from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_groq import ChatGroq
from langchain.chains import RetrievalQA




In [ ]:
# ================== ROBUST LOADERS ==================
class RobustPDFLoader(PyPDFLoader):
    """Try PyMuPDF first, then fall back to PyPDFLoader."""
    def load(self):
        try:
            doc = fitz.open(self.file_path)
            pages = []
            for i, page in enumerate(doc):
                text = page.get_text("text") or ""
                if text.strip():
                    pages.append(Document(
                        page_content=text,
                        metadata={
                            "source": str(self.file_path),
                            "page": i+1,
                            "loader": "pymupdf"
                        }
                    ))
            doc.close()
            if pages:
                return pages
        except Exception as e:
            print(f"[PDF] PyMuPDF failed for {self.file_path}: {e}\n{traceback.format_exc()}")
        try:
            return super().load()
        except Exception as e:
            print(f"[PDF] PyPDFLoader failed for {self.file_path}: {e}\n{traceback.format_exc()}")
            return []


def load_excel_to_documents(file_path: str) -> list[Document]:
    docs = []
    try:
        xls = pd.ExcelFile(file_path)
        for sheet_name in xls.sheet_names:
            df = xls.parse(sheet_name)
            text = df.to_csv(index=False)
            if text.strip():
                docs.append(Document(
                    page_content=text,
                    metadata={
                        "source": str(file_path),
                        "sheet": sheet_name,
                        "loader": "pandas_excel"
                    }
                ))
    except Exception as e:
        print(f"[EXCEL] Failed {file_path}: {e}\n{traceback.format_exc()}")
    return docs


def load_file_to_documents(file_path: str) -> list[Document]:
    """Load a single file into Documents depending on type."""
    ext = Path(file_path).suffix.lower()
    try:
        if ext == ".pdf":
            return RobustPDFLoader(file_path).load()
        elif ext in [".txt", ".md", ".log"]:
            return TextLoader(file_path, autodetect_encoding=True).load()
        elif ext == ".csv":
            return CSVLoader(file_path).load()
        elif ext in [".xls", ".xlsx"]:
            return load_excel_to_documents(file_path)
        elif ext == ".docx":
            return Docx2txtLoader(file_path).load()
        else:
            print(f"[SKIP] Unsupported file type: {file_path}")
            return []
    except Exception as e:
        print(f"[LOAD ERR] {file_path}: {e}\n{traceback.format_exc()}")
        return []

def gather_documents_recursive(root_dir: str, allowed_ext=None) -> list[Document]:
    """Walk directory recursively, load all supported files into Documents."""
    if allowed_ext is None:
        allowed_ext = {".pdf", ".txt", ".csv", ".xls", ".xlsx", ".docx"}
    all_docs = []
    for dirpath, _, filenames in os.walk(root_dir):
        for name in filenames:
            ext = Path(name).suffix.lower()
            if ext in allowed_ext:
                fpath = os.path.join(dirpath, name)
                docs = load_file_to_documents(fpath)
                for d in docs:
                    d.metadata.setdefault("source", fpath)
                    d.metadata["relpath"] = os.path.relpath(fpath, root_dir)
                    d.metadata["subfolder"] = os.path.relpath(dirpath, root_dir)
                all_docs.extend(docs)
    return all_docs
# ================== RAG PIPELINE ==================
def build_vectorstore(root_dir: str, embeddings_model="sentence-transformers/all-MiniLM-L6-v2"):
    print(f"[INFO] Loading documents from {root_dir} ...")
    docs = gather_documents_recursive(root_dir)

    print(f"[INFO] Loaded {len(docs)} raw documents. Splitting ...")
    splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    chunks = splitter.split_documents(docs)

    print(f"[INFO] Split into {len(chunks)} chunks. Building FAISS index ...")
    embeddings = HuggingFaceEmbeddings(model_name=embeddings_model)
    vectorstore = FAISS.from_documents(chunks, embeddings)

    return vectorstore


def build_rag_chain(vectorstore, groq_api_key: str, model="llama-3.3-70b-versatile"):
    llm = ChatGroq(api_key=groq_api_key, model=model)
    retriever = vectorstore.as_retriever(search_kwargs={"k": 5})
    qa_chain = RetrievalQA.from_chain_type(
        llm=llm,
        retriever=retriever,
        chain_type="stuff",
        return_source_documents=True
    )
    return qa_chain


def rag_query(qa_chain, query: str):
    result = qa_chain.invoke({"query": query})
    print("\n=== ANSWER ===")
    print(result["result"])

    print("\n=== SOURCES ===")
    for doc in result["source_documents"]:
        print(f"- {doc.metadata.get('relpath', doc.metadata.get('source'))}")
    return result


# ================== USAGE EXAMPLE ==================
# ================== USAGE EXAMPLE ==================
if __name__ == "__main__":
    DATA_DIR = ""   # <-- adjust path in Kaggle
    GROQ_API_KEY = ""                # <-- don't hardcode, use Kaggle secrets if possible

    # Build Vectorstore
    vs = build_vectorstore(DATA_DIR)

    # Create RAG Chain
    rag_chain = build_rag_chain(vs, GROQ_API_KEY)

    # Query Example
    rag_query(rag_chain, "What are the approved insecticides in India?")


[INFO] Loading documents from /kaggle/input/ai-rag/ai_farmer_database ...
[INFO] Loaded 47232 raw documents. Splitting ...
[INFO] Split into 71116 chunks. Building FAISS index ...

=== ANSWER ===
The provided context does not give a comprehensive list of approved insecticides in India. However, it does mention some insecticides that are approved for use in certain contexts:

1. **For Cotton Kapas (Fiber Crops)**:
   - Oxydemeton methyl 25EC
   - Dimethoate 30EC
   - Acetamiprid 20SP
   - Confidor (likely referring to Imidacloprid)
   - Monocrotophos
   - Spinosad 45 SC
   - Takumi (dosage mentioned but not the chemical composition)
   - Deltamethrin 25 EC
   - Cypermethrin 10 EC

2. **General Mention**:
   - The context refers to a document that lists insecticides registered under the Insecticides Act, 1968, up to 30/06/2020. It mentions that detailed information on insecticides registered for agricultural use, public health use, and household use can be found on specific page numbers,

In [13]:
# Query Example
rag_query(rag_chain, "What are the approved pesticide for potato brown rot")



=== ANSWER ===
Based on the provided context, the approved pesticides for potato brown rot are:

1. Copper oxychloride (25g/liter of water) 
2. IndoFil-M-45 (2g/ml of water)
3. Hexaconazole 5 EC (1 liter/ha with irrigation) 

Additionally, some recommendations include:

* Removing and destroying affected plants
* Providing proper drainage facilities
* Using soil drenching with copper oxychloride
* Selecting brown rot disease-free tubers.

=== SOURCES ===
- KCC/Potato/Potato_Brown_rot.xlsx
- KCC/Potato/Potato_Early_blight.xlsx
- KCC/Potato/Potato_Thrips.xlsx
- KCC/Potato/Potato_Early_blight.xlsx
- KCC/Potato/Potato_Early_blight.xlsx


{'query': 'What are the approved pesticide for potato brown rot',
 'result': 'Based on the provided context, the approved pesticides for potato brown rot are:\n\n1. Copper oxychloride (25g/liter of water) \n2. IndoFil-M-45 (2g/ml of water)\n3. Hexaconazole 5 EC (1 liter/ha with irrigation) \n\nAdditionally, some recommendations include:\n\n* Removing and destroying affected plants\n* Providing proper drainage facilities\n* Using soil drenching with copper oxychloride\n* Selecting brown rot disease-free tubers.',
 'source_documents': [Document(id='c52a38b2-2577-4fa5-ad44-1091ab8594d0', metadata={'source': '/kaggle/input/ai-rag/ai_farmer_database/KCC/Potato/Potato_Brown_rot.xlsx', 'sheet': 'Potato_Brown_rot', 'loader': 'pandas_excel', 'relpath': 'KCC/Potato/Potato_Brown_rot.xlsx', 'subfolder': 'KCC/Potato'}, page_content='Vegetables,12,Potato,Plant Protection,RABI,BIHAR,Brown rot,Select brown rot disease free tubers Give proper drainage facilities  Remove and destroy the affected plants\

In [14]:
# Query Example
rag_query(rag_chain, "What are the approved pesticide for brinjal")


=== ANSWER ===
I don't have information about approved pesticides for brinjal. The provided context only mentions whitefly management in brinjal in the context of cotton, with recommendations for sprays such as:

- Akkni asthiram 300 ml/10 lit of water
- Imidacloprid 0.5 ml/lit of water

However, these recommendations are not explicitly stated as approved pesticides for brinjal. For accurate and up-to-date information on approved pesticides for brinjal, I recommend consulting a reliable source or a relevant agricultural authority.

=== SOURCES ===
- KCC/paddy/Paddy_Dhan_hispa.xlsx
- KCC/paddy/Paddy_Dhan_Sheath_rot.xlsx
- KCC/Cotton/Cotton_Kapas_Whitefly.xlsx
- KCC/paddy/Paddy_Dhan_hispa.xlsx
- KCC/paddy/Paddy_Dhan_hispa.xlsx


{'query': 'What are the approved pesticide for brinjal',
 'result': "I don't have information about approved pesticides for brinjal. The provided context only mentions whitefly management in brinjal in the context of cotton, with recommendations for sprays such as:\n\n- Akkni asthiram 300 ml/10 lit of water\n- Imidacloprid 0.5 ml/lit of water\n\nHowever, these recommendations are not explicitly stated as approved pesticides for brinjal. For accurate and up-to-date information on approved pesticides for brinjal, I recommend consulting a reliable source or a relevant agricultural authority.",
 'source_documents': [Document(id='1ea58bf0-097d-480b-988b-772caec2f2e1', metadata={'source': '/kaggle/input/ai-rag/ai_farmer_database/KCC/paddy/Paddy_Dhan_hispa.xlsx', 'sheet': 'Paddy_Dhan_hispa', 'loader': 'pandas_excel', 'relpath': 'KCC/paddy/Paddy_Dhan_hispa.xlsx', 'subfolder': 'KCC/paddy'}, page_content=',Cereals,,10,,Paddy Dhan,,Plant Protection,,,ODISHA,RICE HISPA,RECOMENDED DOSE OF QUINALPHO

In [ ]:
# Query Example
rag_query(rag_chain, "What are the approved pesticide for brinjal")

In [15]:
rag_query(rag_chain, "What are the approved pesticide for banana")


=== ANSWER ===
Based on the context provided, the following are some of the approved pesticides for banana:

1. Mancozeb 75% WP @600-800 g in 400 l of water/acre (for Sigatoka disease)
2. Propiconazole 25% EC @ 0.1% (100 ml/100 l water.) in 200-400 l of water/acre (for Sigatoka disease)
3. Copper oxychloride 50% WP @ 1Kg in 300-400 l of water/acre (for Sigatoka disease)
4. Methyl demeton 25 EC or Dimethoate 30 EC 2ml per litre of water (for Aphids infestation)

Note: The dosage and usage of these pesticides may vary depending on the specific pest and disease management requirements. It's always best to consult with a local agricultural expert or follow the recommended guidelines for pesticide usage in your region.

=== SOURCES ===
- IPM/Banana.pdf
- IPM/Banana.pdf
- IPM/Banana.pdf
- KCC/banana/Banana_Aphids.csv
- KCC/banana/Banana_Aphids.csv


{'query': 'What are the approved pesticide for banana',
 'result': "Based on the context provided, the following are some of the approved pesticides for banana:\n\n1. Mancozeb 75% WP @600-800 g in 400 l of water/acre (for Sigatoka disease)\n2. Propiconazole 25% EC @ 0.1% (100 ml/100 l water.) in 200-400 l of water/acre (for Sigatoka disease)\n3. Copper oxychloride 50% WP @ 1Kg in 300-400 l of water/acre (for Sigatoka disease)\n4. Methyl demeton 25 EC or Dimethoate 30 EC 2ml per litre of water (for Aphids infestation)\n\nNote: The dosage and usage of these pesticides may vary depending on the specific pest and disease management requirements. It's always best to consult with a local agricultural expert or follow the recommended guidelines for pesticide usage in your region.",
 'source_documents': [Document(id='080f91a5-9feb-426e-8de7-adaea6de564b', metadata={'source': '/kaggle/input/ai-rag/ai_farmer_database/IPM/Banana.pdf', 'page': 27, 'loader': 'pymupdf', 'relpath': 'IPM/Banana.pdf', 

In [16]:
# Query Example
rag_query(rag_chain, "CONTROL OF RED RUST IN MANGO")


=== ANSWER ===
Based on the provided context, the control of Red Rust in Mango can be achieved through the following methods:

1. **Copper Oxychloride 50 WP**: This is a commonly recommended fungicide for controlling Red Rust in Mango. The dosage varies, but common recommendations include:
	* 2-3 grams per liter of water
	* 25 grams per liter of water
	* 500 grams per 200 liters of water per acre
	* 600 grams per 200 liters of water per acre
2. **Spraying**: Spray the trees with the recommended fungicide solution, making sure to cover all affected areas.
3. **Pruning**: Prune off diseased or dead shoots to prevent the spread of the disease.
4. **Bordeaux Paste or Copper Oxychloride paste**: Apply these pastes to the cut ends of pruned shoots to prevent further infection.
5. **Mancozeb, Carbendazim, or other fungicides**: These can be used as alternative or additional fungicides for controlling Red Rust in Mango.
6. **Repeat sprays**: Repeat sprays at fortnightly intervals (every 10-14

{'query': 'CONTROL OF RED RUST IN MANGO',
 'result': "Based on the provided context, the control of Red Rust in Mango can be achieved through the following methods:\n\n1. **Copper Oxychloride 50 WP**: This is a commonly recommended fungicide for controlling Red Rust in Mango. The dosage varies, but common recommendations include:\n\t* 2-3 grams per liter of water\n\t* 25 grams per liter of water\n\t* 500 grams per 200 liters of water per acre\n\t* 600 grams per 200 liters of water per acre\n2. **Spraying**: Spray the trees with the recommended fungicide solution, making sure to cover all affected areas.\n3. **Pruning**: Prune off diseased or dead shoots to prevent the spread of the disease.\n4. **Bordeaux Paste or Copper Oxychloride paste**: Apply these pastes to the cut ends of pruned shoots to prevent further infection.\n5. **Mancozeb, Carbendazim, or other fungicides**: These can be used as alternative or additional fungicides for controlling Red Rust in Mango.\n6. **Repeat sprays**

In [17]:
# Query Example
rag_query(rag_chain, "farmer want to know about control of red rust in mango")


=== ANSWER ===
The recommended control measure for red rust in mango is to spray Copper Oxychloride 50 WP. The dosage varies, but some common recommendations are:

* 600 g per 200 liters of water per 1 acre of land (recommended in Odisha)
* 2-3 grams per liter of water (recommended in Uttar Pradesh)
* 25 grams per liter of water (recommended in Uttar Pradesh)
* 500 grams per 200 liters of water (recommended in Uttar Pradesh)

It's also recommended to repeat the spray if symptoms appear again after 10 days of spray. Additionally, some other measures that can be taken include pruning off diseased or dead shoots and applying Bordeaux Paste or Copper oxychloride paste on cut ends.

It's worth noting that different states have slightly different recommendations, so it's best to consult with local agricultural experts for specific advice.

=== SOURCES ===
- KCC/Mango/Mango_Red_rust.xlsx
- KCC/Mango/Mango_Red_rust.xlsx
- KCC/Mango/Mango_Red_rust.xlsx
- KCC/Mango/Mango_Red_rust.xlsx
- KCC/Man

{'query': 'farmer want to know about control of red rust in mango',
 'result': "The recommended control measure for red rust in mango is to spray Copper Oxychloride 50 WP. The dosage varies, but some common recommendations are:\n\n* 600 g per 200 liters of water per 1 acre of land (recommended in Odisha)\n* 2-3 grams per liter of water (recommended in Uttar Pradesh)\n* 25 grams per liter of water (recommended in Uttar Pradesh)\n* 500 grams per 200 liters of water (recommended in Uttar Pradesh)\n\nIt's also recommended to repeat the spray if symptoms appear again after 10 days of spray. Additionally, some other measures that can be taken include pruning off diseased or dead shoots and applying Bordeaux Paste or Copper oxychloride paste on cut ends.\n\nIt's worth noting that different states have slightly different recommendations, so it's best to consult with local agricultural experts for specific advice.",
 'source_documents': [Document(id='2557ea7b-6bdc-41d4-bd13-267cf3a2dafe', metad

In [18]:
rag_query(rag_chain,"what can I do for Citrus Greening")


=== ANSWER ===
I don't have any information about Citrus Greening in the provided context. The context only discusses diseases related to mango trees, such as Brown eye spot disease, Black rot, Berry blotch, Dieback/Anthracnose, and Root rot, as well as Red rust management in mango. It does not mention Citrus Greening. If you're looking for information on Citrus Greening, I recommend consulting a different source or expert in the field of citrus tree care.

=== SOURCES ===
- IPM/Coffee.pdf
- KCC/Mango/Mango_Red_rust.xlsx
- KCC/Mango/Mango_Red_rust.xlsx
- IPM/Crop protection under IPM.pdf
- KCC/Mango/Mango_Red_rust.xlsx


{'query': 'what can I do for Citrus Greening',
 'result': "I don't have any information about Citrus Greening in the provided context. The context only discusses diseases related to mango trees, such as Brown eye spot disease, Black rot, Berry blotch, Dieback/Anthracnose, and Root rot, as well as Red rust management in mango. It does not mention Citrus Greening. If you're looking for information on Citrus Greening, I recommend consulting a different source or expert in the field of citrus tree care.",
 'source_documents': [Document(id='82603ca8-4d89-4974-9c78-14ef54aefa71', metadata={'source': '/kaggle/input/ai-rag/ai_farmer_database/IPM/Coffee.pdf', 'page': 22, 'loader': 'pymupdf', 'relpath': 'IPM/Coffee.pdf', 'subfolder': 'IPM'}, page_content='\uf0b7 \nMake a trench around the trees to depth of 30 cm and make sure the \nsoil should be inside the circle.   \n \nBrown eye \nspot \ndisease  \n \nCultural control: \n\uf0b7 \nBrown eye disease is associated with a lack of in  particular n

In [19]:
rag_query(rag_chain, "What are the approved pesticide for cordana disease in banana")


=== ANSWER ===
I don't know the approved pesticide for Cordana disease in banana. The provided context does not mention Cordana disease or its control measures.

=== SOURCES ===
- IPM/Banana.pdf
- IPM/Banana.pdf
- KCC/banana/Banana_Moko_disease.csv
- KCC/banana/Banana_stem_borer.csv
- KCC/banana/Banana_Panama_disease.csv


{'query': 'What are the approved pesticide for cordana disease\xa0in\xa0banana',
 'result': "I don't know the approved pesticide for Cordana disease in banana. The provided context does not mention Cordana disease or its control measures.",
 'source_documents': [Document(id='080f91a5-9feb-426e-8de7-adaea6de564b', metadata={'source': '/kaggle/input/ai-rag/ai_farmer_database/IPM/Banana.pdf', 'page': 27, 'loader': 'pymupdf', 'relpath': 'IPM/Banana.pdf', 'subfolder': 'IPM'}, page_content='AESA based IPM – Banana\n19\n• \nThe whole fruit bunches may be bagged inside banana leaves at later stage.\n• \nThe collection and destruction of fallen, damaged and overripe fruits is strongly recommended to reduce \nthe resident population of fruit fl ies. \n• \nFruit fl y trap can be erected at every 20 meter distance.\n• \nEarly harvesting may also reduce the fruit infestation and damage.\nBag worm \n• \nThe sanitary measures like hand picking and destroying the infested leaf parts should be adopted.

In [20]:
rag_query(rag_chain, "CONTROL OF RED RUST IN MANGO")


=== ANSWER ===
Based on the provided context, the control of Red Rust in Mango can be achieved through the following methods:

1. **Copper Oxychloride 50 WP**: This is a widely recommended fungicide for controlling Red Rust in Mango. The dosage varies, but common recommendations include:
	* 2-3 grams per liter of water
	* 25 grams per liter of water
	* 500 grams per 200 liters of water per acre
	* 600 grams per 200 liters of water per acre
2. **Spraying**: Spray the fungicide on the affected mango trees, making sure to cover all parts of the tree.
3. **Pruning**: Prune off diseased or dead shoots to prevent the spread of the disease.
4. **Bordeaux Paste or Copper Oxychloride Paste**: Apply these pastes on cut shoots to prevent infection.
5. **Mancozeb, Carbendazim, or other fungicides**: These can be used as alternative or additional treatments, but Copper Oxychloride 50 WP is the most commonly recommended.

It's essential to note that the dosage and application method may vary depend

{'query': 'CONTROL OF RED RUST IN MANGO',
 'result': "Based on the provided context, the control of Red Rust in Mango can be achieved through the following methods:\n\n1. **Copper Oxychloride 50 WP**: This is a widely recommended fungicide for controlling Red Rust in Mango. The dosage varies, but common recommendations include:\n\t* 2-3 grams per liter of water\n\t* 25 grams per liter of water\n\t* 500 grams per 200 liters of water per acre\n\t* 600 grams per 200 liters of water per acre\n2. **Spraying**: Spray the fungicide on the affected mango trees, making sure to cover all parts of the tree.\n3. **Pruning**: Prune off diseased or dead shoots to prevent the spread of the disease.\n4. **Bordeaux Paste or Copper Oxychloride Paste**: Apply these pastes on cut shoots to prevent infection.\n5. **Mancozeb, Carbendazim, or other fungicides**: These can be used as alternative or additional treatments, but Copper Oxychloride 50 WP is the most commonly recommended.\n\nIt's essential to note 

In [21]:
rag_query(rag_chain, "farmer want to know about control of red rust in mango")


=== ANSWER ===
The recommended control measure for Red Rust in Mango is to spray Copper Oxychloride 50 WP. The dosage varies, but some common recommendations are:

* 600 g per 200 liters of water per 1 acre of land (recommended in Odisha)
* 2-3 grams per liter of water (recommended in Uttar Pradesh)
* 25 grams per liter of water (recommended in Uttar Pradesh)
* 2-25 ml per liter of water (recommended in Uttar Pradesh)

It's also recommended to repeat the spray if symptoms appear again after 10 days of the initial spray. Additionally, some other measures that can be taken include pruning off diseased or dead shoots and applying Bordeaux Paste or Copper oxychloride paste on the cut. 

It's always best to consult with a local agricultural expert or authority for specific guidance tailored to your region and situation.

=== SOURCES ===
- KCC/Mango/Mango_Red_rust.xlsx
- KCC/Mango/Mango_Red_rust.xlsx
- KCC/Mango/Mango_Red_rust.xlsx
- KCC/Mango/Mango_Red_rust.xlsx
- KCC/Mango/Mango_Red_rust.

{'query': 'farmer want to know about control of red rust in mango',
 'result': "The recommended control measure for Red Rust in Mango is to spray Copper Oxychloride 50 WP. The dosage varies, but some common recommendations are:\n\n* 600 g per 200 liters of water per 1 acre of land (recommended in Odisha)\n* 2-3 grams per liter of water (recommended in Uttar Pradesh)\n* 25 grams per liter of water (recommended in Uttar Pradesh)\n* 2-25 ml per liter of water (recommended in Uttar Pradesh)\n\nIt's also recommended to repeat the spray if symptoms appear again after 10 days of the initial spray. Additionally, some other measures that can be taken include pruning off diseased or dead shoots and applying Bordeaux Paste or Copper oxychloride paste on the cut. \n\nIt's always best to consult with a local agricultural expert or authority for specific guidance tailored to your region and situation.",
 'source_documents': [Document(id='2557ea7b-6bdc-41d4-bd13-267cf3a2dafe', metadata={'source': '/k

In [22]:
rag_query(rag_chain,"what can I do for Citrus Greening") # it does not have these information in my pdf or documents


=== ANSWER ===
I don't know the answer to that question based on the provided context. The context only discusses various diseases affecting mango trees, such as Brown eye spot disease, Black rot, Berry blotch, Dieback/Anthracnose, and Root rot, as well as Red rust management in Mango. There is no information provided about Citrus Greening.

=== SOURCES ===
- IPM/Coffee.pdf
- KCC/Mango/Mango_Red_rust.xlsx
- KCC/Mango/Mango_Red_rust.xlsx
- IPM/Crop protection under IPM.pdf
- KCC/Mango/Mango_Red_rust.xlsx


{'query': 'what can I do for Citrus Greening',
 'result': "I don't know the answer to that question based on the provided context. The context only discusses various diseases affecting mango trees, such as Brown eye spot disease, Black rot, Berry blotch, Dieback/Anthracnose, and Root rot, as well as Red rust management in Mango. There is no information provided about Citrus Greening.",
 'source_documents': [Document(id='82603ca8-4d89-4974-9c78-14ef54aefa71', metadata={'source': '/kaggle/input/ai-rag/ai_farmer_database/IPM/Coffee.pdf', 'page': 22, 'loader': 'pymupdf', 'relpath': 'IPM/Coffee.pdf', 'subfolder': 'IPM'}, page_content='\uf0b7 \nMake a trench around the trees to depth of 30 cm and make sure the \nsoil should be inside the circle.   \n \nBrown eye \nspot \ndisease  \n \nCultural control: \n\uf0b7 \nBrown eye disease is associated with a lack of in  particular nitrogen \nand a lack of shade \n\uf0b7 \nImprovement of tree nutrition and shading should be sufficient to \ncontrol t